In [10]:
from numpy import linalg as LA
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import time
import os

## 目前進度
### 還須找到更好的擬合公式
### 或是找到適合 sample 的距離 

## 縮寫頓應
pt   -> points  <br>
dist -> distance <br>
spl  -> sample


## 影像處理

In [13]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return blur, nlm, otsu, edge

def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## Get subpixel edge locations

In [3]:
def sigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a*(x-b)))

In [ ]:
def get_influential_neighbors():
    pass

In [ ]:
def get_negihbors_val(neighbors_loc, img):
    neighbors_val = [img[y,x] for y, x in neighbors_loc]
    return neighbors_val

In [ ]:
def get_negihbors_wt(coord, neighbors_loc):
    neighbors_dist = LA.norm(coord - neighbors_loc, axis=1)
    
    ## 想辦法使得以距離違反比且對於剛好在pixel上時回傳整數值
    pass

In [21]:
a = np.asarray([[-1,2],[3,4]])

LA.norm(a, axis=1)

array([2.23606798, 5.        ])

In [ ]:
def get_sample_value(coord, img):
    ## 取得有影響力的 4 個相鄰點及相對應的 值 和 權重
    neighbors_loc = get_influential_neighbors(coord)
    neighbors_val = get_negihbors_val(neighbors_loc, img)
    neighbors_wt  = get_negihbors_wt(coord, neighbors_loc)
    
    ## 計算 sample 的值
    value = np.dot(neighbors_val, neighbors_wt)
    return value
    

In [4]:
### Stop
def get_sample_value(coord, img):
    center = np.around(coord)
    
    x, y = int(center[0]), int(center[1])
    neighbor_val = []
    neighbor_dist = []
    
    ## 取的9格的距離跟值
    for i in range(-1,2):
        for j in range(-1,2):
            dist = LA.norm((center + np.asarray([j,i])) - coord)
            neighbor_val.append(img[y+i, x+j])
            neighbor_dist.append(dist)

    ## 依照距離反比給權重
    neighbor_dist = np.asarray(neighbor_dist)/np.sum(neighbor_dist)
    neighbor_dist = (1 - np.asarray(neighbor_dist))/8
    
    value = np.dot(neighbor_val, neighbor_dist)
    return value
### Stop

In [12]:
def get_subpixel_edges(center, edge_pts, img):
    sp_edge = []
    
    for pt in edge_pts:
        ## 取得中心到邊緣的單位向量
        c2e_Vector = pt - center
        c2e_Vector = c2e_Vector/LA.norm(c2e_Vector)
        
        
        spl_pts = []
        spl_vals = []
        for i in range(-10, 11):
            ## 生出要擬合 S 曲線的點
            spl_loc = pt + i*c2e_Vector              # 需要再找道合適參數
            spl_val = get_sample_value(spl_loc, img)
            
            spl_pts.append(spl_loc)
            spl_vals.append(spl_val)
            

#         print(spl_pts)
#         print(spl_vals)
        
        x_label = np.asarray(range(0,21))
        

        ## 擬合 S 曲線
        popt, pcov = curve_fit(sigmoid, x_label, spl_vals)
#         print(popt)
        
        x = np.linspace(-5, 35, 50)
        y = sigmoid(x, *popt)

#         plt.plot(x_label, spl_vals, 'o', label='data')
# #         plt.plot(x,y, label='fit')
#         plt.ylim(100, 140)
#         plt.legend(loc='best')
#         plt.show()
        
#         break
    
    return sp_edge

In [14]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
# os.chdir("/home/mj/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file = "10_mean_80.bmp"
img = cv2.imread(file, 0)

time_start = time.time()

blur, nlm, otsu, edge = image_processing(img)
edge_points = get_edge_points(edge)
cx,cy,r,_ = cf.least_squares_circle((edge_points))
sp_edges = get_subpixel_edges(np.asarray([cx,cy]), edge_points, nlm)


elapsed_time = time.time() - time_start
print(elapsed_time)
# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# print("{}, {}".format(cx, cy))

1.0810234546661377
